# TP : Text segmentation using Hidden Markov Models


## Author:

- Gabriele LORENZO
- Aldo PIETROMATERA


## Questions:


### Q.1. Give the value of the $\pi$ vector of the initial probabilities:

The initial probabilities vector $\pi$ will be:

$$\pi = \begin{bmatrix} 1 \\ 0 \end{bmatrix}$$

This is beacuse the first state is always the start state.


### Q.2. : What is the probability to move from state 1 to state 2 What is the probability to remain in state 2? What is the lower/higher probability? Try to explain why.

The transition matrix $A$ is:

$$A = \begin{pmatrix} 0.999218078035812 & 0.000781921964187974 \\ 0 & 1 \end{pmatrix}$$

The probability to move from state 2 to state 1 is of course 0, because once we are in the body of the email it is impossible to go back to the header. Hence the probability to remain in state 2 is 1.

The lower probability is the one to move from state 1 to state 2 (0.000781921964187974), because it is very unlikely to move from the header to the body of the email.


### Q.3. What is the size of B?

The size of B is $256 \times 2$, because we have 2 states and 256 possible observations. Each row of B contains the probabilities of observing a certain character _c_ in a state _s_.


In [1]:
import os
import glob
import numpy as np

In [2]:
ROOT = os.path.abspath(".")

DISTR_DIR = os.path.join(ROOT, "distribution")
RES_DIR = os.path.join(ROOT, "res")
DATA_DIR = os.path.join(ROOT, "dat")
SEGMENTED_DIR = os.path.join(ROOT, "segmented")

### Coding/Decoding Mails


In [3]:
# Iterate through files and load the text
def files_iter(data_dir, with_filename=False):
    files = glob.glob("{}/*.dat".format((data_dir)))

    if with_filename:
        for f in files:
            # Get the filename
            filename = f.split("\\")[-1].split(".")[0]
            # Return filename and associated text
            yield filename, open(f, "r").read()
    else:
        for f in files:
            yield open(f, "r").read()

In [4]:
# And we get a generator that will allow us to iterate through the mails
mail_iter = files_iter(DATA_DIR, with_filename=True)

### Distribution files


In [5]:
# Writing a function to get the probability data
def get_emission_prob(p_dir):
    return np.loadtxt(p_dir, delimiter="\t", dtype=float)

In [6]:
# Inputs to the Viterbi function
trans = np.array([[0.999218078035812, 0.000781921964187974], [0, 1]])
emission_prob = get_emission_prob(os.path.join(DISTR_DIR, "P.text"))
states = np.array([1, 2])
start_prob = np.array([1, 0])

### To implement:


In [7]:
# Viterbi function
def viterbi(obs, states, start_prob, trans, emission_prob):
    """
    Viterbi Algorithm Implementation

    Keyword arguments:
        - obs: sequence of observation
        - states: list of states
        - start_prob: vector of the initial probabilities
        - trans: transition matrix
        - emission_prob: emission probability matrix
    Returns:
        - seq: sequence of state
    """

    # Avoid underflow: use the logarithm !
    # Avoid 0 in logarithm: use a small constant !
    small = np.finfo(np.float64).tiny

    start_prob = np.log(start_prob + small)
    trans = np.log(trans + small)
    emission_prob = np.log(emission_prob + small)

    obs = [int(x) for x in obs.split("\n") if x != ""]

    # Number of observations
    T = len(obs)
    # Number of model states
    N = len(states)

    # Initialisation
    log_l = np.zeros((T, N))
    bcktr = np.zeros((T, N))

    # Viterbi

    # Forward loop:
    log_l[0, :] = start_prob + emission_prob[obs[0], :]

    for t in range(1, T):
        for s in range(N):
            log_l[t, s] = (
                np.max(log_l[t - 1, :] + trans[:, s]) + emission_prob[obs[t], s]
            )
            bcktr[t, s] = np.argmax(log_l[t - 1, :] + trans[:, s])

    # Backward loop
    path = np.zeros(T, dtype=int)
    path[-1] = np.argmax(log_l[-1, :])

    for i in range(T - 1, 0, -1):
        path[i - 1] = bcktr[i, path[i]]

    return [states[i] for i in path]

In [8]:
# Creating a directory to put the result of the viterbi function
if not os.path.exists(RES_DIR):
    os.mkdir(RES_DIR)


# Function that will write a viterbi path for a mail in a dedicated result file
def create_viterbi_path_file(mail_name, viterbi_path):
    with open("{}/{}_path.txt".format(RES_DIR, mail_name), "w") as f:
        f.write("".join([str(c) for c in viterbi_path]))

In [9]:
# Using our generator, we get the mail names and data
for name_file, data in mail_iter:
    # Find out the viterbi path using viterbi
    viterbi_path = viterbi(data, states, start_prob, trans, emission_prob)
    # Put it in the result file
    create_viterbi_path_file(name_file, viterbi_path)

### Visualizing segmentation


In [10]:
def visualize_segmentation(mail_filename, visualized_mail_filename, path):
    ## @parameter mail_filename : Path to the mail on wich we try the algorithm.
    ## @parameter visualized_mail_filename : The path on which we write the mail with the v
    ## @parameter path : The sequence of 0 and 1 that the Viterbi algorithm returns.
    ## return: True if the header corresponds to the
    visu = open(visualized_mail_filename, "w")

    if mail_filename.endswith(".dat"):
        mail_filename = mail_filename[:-4] + ".txt"

    mail = open(mail_filename, "r")
    mail_content = mail.read()
    idx_split = len(path) - path.sum() - 1
    visu.write(mail_content[:idx_split])
    visu.write("\n===================== cut here\n")
    visu.write(mail_content[idx_split:])
    visu.close()
    mail.close()

    return idx_split

In [11]:
# Writing a function getting the original mail, the result of viterbi, and applying the segmentation script
# Then putting the result
def segment_mail(mail_name, data_dir, res_dir):
    # Get the full path of the mail
    mail_filename = os.path.join(data_dir, f"{mail_name}.txt")
    # Get the full path of the result
    path_file = os.path.join(res_dir, f"{mail_name}_path.txt")

    # Read the result
    with open(path_file, "r") as f:
        path_read = f.read()
        path = [int(p) - 1 for p in path_read]

    path = np.array(path)

    # Execute the visualization script
    segmented_filename = os.path.join(SEGMENTED_DIR, f"{mail_name}_segmented.txt")
    idx_split = visualize_segmentation(mail_filename, segmented_filename, path)
    print(f"Mail {mail_name} segmented at index {idx_split} (out of {len(path)})")
    # Get the results
    formatted_mail_text = open(segmented_filename, "r").read()
    # Go through the resulting text until the cutting line
    cutting_line = "\n===================== cut here\n"

    if cutting_line in formatted_mail_text:
        # If this was not the last line, return the text cut in to parts: header and body
        header, body = formatted_mail_text.split(cutting_line, 1)
        return header, body
    else:
        # If not, it's just a header
        return formatted_mail_text, None

In [12]:
# Getting mails names
mail_names = [name for name, _ in files_iter(DATA_DIR, with_filename=True)]

if not os.path.exists(SEGMENTED_DIR):
    os.mkdir(SEGMENTED_DIR)

# Call the function and look at the result of segmentation
for mail_name in mail_names:
    header, body = segment_mail(mail_name, DATA_DIR, RES_DIR)
    print("Mail:", mail_name)
    #print("Header:\n", header)
    #print("#" * 50)
    #print("Body:\n", body)
    #print("\n" * 5)

Mail mail1 segmented at index 3795 (out of 5216)
Mail: mail1
Mail mail10 segmented at index 2845 (out of 3715)
Mail: mail10
Mail mail11 segmented at index 2850 (out of 3475)
Mail: mail11
Mail mail12 segmented at index 2937 (out of 3993)
Mail: mail12
Mail mail13 segmented at index 2303 (out of 3328)
Mail: mail13
Mail mail14 segmented at index 4812 (out of 6576)
Mail: mail14
Mail mail15 segmented at index 2182 (out of 6808)
Mail: mail15
Mail mail16 segmented at index 1970 (out of 2627)
Mail: mail16
Mail mail17 segmented at index 2281 (out of 3425)
Mail: mail17
Mail mail18 segmented at index 2366 (out of 3077)
Mail: mail18
Mail mail19 segmented at index 2100 (out of 2620)
Mail: mail19
Mail mail2 segmented at index 2444 (out of 3376)
Mail: mail2
Mail mail20 segmented at index 1839 (out of 2434)
Mail: mail20
Mail mail21 segmented at index 2102 (out of 2664)
Mail: mail21
Mail mail22 segmented at index 2233 (out of 3643)
Mail: mail22
Mail mail23 segmented at index 2167 (out of 3750)
Mail: mai

### Q.4. Print the track and present and discuss the results obtained on mail11.txt to mail30.txt.

We are going to take as example the mail11.txt and mail30.txt files.

For email 11, the segmentation is done at the 2850th character. If we look at the real email, we can see that the header ends at the 2851th character, so the segmentation is almost correct.

For email 30, the segmentation is done at the 2172th character. If we look at the real email, we can see that the header ends at the 2250th character. Even if the segmentation is not perfect, it is still very close to the real value.


### Q.5. How would you model the problem if you had to segment the mails in more than two parts (for example : header, body, signature)? Draw a diagram of the corresponding Hidden Markov model and give an example of A matrix that would be suitable in this case.

Let's take the header, body and signature as the three states of the model. The transition matrix $A$ would be:

$$A = \begin{pmatrix} a_{11} & a_{12} & a_{13} \\ 0 & a_{22} & a_{23} \\ 0 & 0 & 1 \end{pmatrix}$$

As we can see, it is impossible to move from the body to the header or the signature, and it is impossible to move from the signature to the header or the body.
The value of $a_{13}$ would be non-zero if we want to allow the possibility of moving from the header to the signature, otherwise it would be 0.

The initial probabilities vector $\pi$ would be:

$$\pi = \begin{bmatrix} 1 \\ 0 \\ 0 \end{bmatrix}$$

The emission matrix $B$ would be a $256 \times 3$ matrix, because we have 3 states and 256 possible observations.


### Q.6. How would you model the problem of separating the portions of mail included, knowing that they always start with the character ">". Draw a diagram of the corresponding Hidden Markov model.
